In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
from matplotlib.colors import Normalize
import glob
from scipy.interpolate import griddata
from pathlib import Path
import h5py
import sys
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerTuple

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

nsm_plots_postproc = (here / '..' / 'nsm_instabilities').resolve()
sys.path.append(str(nsm_plots_postproc))

import phys_const as pc
import plot_functions as pf
import functions_angular_crossings as fac

### READ DATA FOR TIME DEPENDENT SPECTRUM PLOTS

In [ ]:
with h5py.File(f"/home/erick/software/devscrpts/gw170817_paper_plots/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_10.0m_att_1e-5_energy_spectra_time_evolution.h5", "r") as hf:
    all_f00_Re_split_energy_bins    = hf["all_f00_Re_split_energy_bins"][:]
    all_f00_Rebar_split_energy_bins = hf["all_f00_Rebar_split_energy_bins"][:]
    all_f11_Re_split_energy_bins    = hf["all_f11_Re_split_energy_bins"][:]
    all_f11_Rebar_split_energy_bins = hf["all_f11_Rebar_split_energy_bins"][:]
    all_f22_Re_split_energy_bins    = hf["all_f22_Re_split_energy_bins"][:]
    all_f22_Rebar_split_energy_bins = hf["all_f22_Rebar_split_energy_bins"][:]
    tiempo_s = hf["tiempo_s"][:]
    energybinsMeV = hf["energybinsMeV"][:]

### PLOTING TIME DEPENDENT SPECTRUM

In [ ]:
def plot_colored_lines_grid(x, y_list, time_s, titles, suptitle, filename, ylowerlimit=None, yupperlimit=None):
    fig, axs = plt.subplots(2, 3, figsize=(18, 8), sharex=True, sharey=True, gridspec_kw=dict(hspace=0.0, wspace=0.0))
    axs = axs.flatten()

    norm = Normalize(time_s.min(), time_s.max())
    cmap = plt.cm.copper

    for i, y in enumerate(y_list):
        ax = axs[i]
        for j in range(y.shape[0]):
            color = cmap(norm(time_s[j]))
            ax.plot(x, y[j, :], color=color)

        # Place the title inside the plot, top left corner
        ax.text(0.03, 0.15, titles[i], transform=ax.transAxes, fontsize=22,
            verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
        ax.set_xlim(x.min(), x.max())
        ax.set_ylim(ylowerlimit, yupperlimit)
        ax.tick_params(axis='both', which='both', direction='in', top=True, right=True)
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        # if i % 3 == 0:
        #     ax.set_ylabel(r'$\log(n\, [\mathrm{cm}^{-3}])$')
        # if i >= 3:
        #     ax.set_xlabel(r'$\log(E\, [\mathrm{MeV}])$')

    fig.text(0.5, 0.02, r'$\log_{10}\, E\, [\mathrm{MeV}] $', ha='center', fontsize=22)
    fig.text(0.07, 0.5, r'$\log_{10}\, f $', va='center', rotation='vertical', fontsize=22)

    # Shared colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=axs, location='right', shrink=1.0, pad=0.01)
    cbar.set_label(r'$t \, [\mathrm{ms}]$', fontsize=22)
    cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())

    # Super title
    fig.suptitle(suptitle, fontsize=22)

    # Save and show
    if filename:
        plt.savefig(filename, bbox_inches='tight')
    plt.show()
    plt.close(fig)

a=0
b=-485
c=None

plot_colored_lines_grid(
    x = np.log10(energybinsMeV),
    y_list = [
        np.log10(all_f00_Re_split_energy_bins[a:b:c, :]),
        np.log10(all_f11_Re_split_energy_bins[a:b:c, :]),
        np.log10(all_f22_Re_split_energy_bins[a:b:c, :]),
        np.log10(all_f00_Rebar_split_energy_bins[a:b:c, :]),
        np.log10(all_f11_Rebar_split_energy_bins[a:b:c, :]),
        np.log10(all_f22_Rebar_split_energy_bins[a:b:c, :])
    ],
    time_s = tiempo_s[a:b:c] / 1e-3,
    titles = [
        r'$e$', r'$\mu$', r'$\tau$',
        r'$\bar{e}$', r'$\bar{\mu}$', r'$\bar{\tau}$'
    ],
    suptitle = '',
    filename = f"plots/f_CFI_E_spectrum.pdf",
    ylowerlimit = -9.9,
    yupperlimit = -1
)

In [ ]:
def plot_colored_lines_grid(x, y_list, time_s, titles, suptitle, filename, ylowerlimit=None, yupperlimit=None):
    fig, axs = plt.subplots(2, 3, figsize=(18, 8), sharex=True, sharey=True, gridspec_kw=dict(hspace=0.0, wspace=0.0))
    axs = axs.flatten()

    norm = Normalize(time_s.min(), time_s.max())
    cmap = plt.cm.copper

    for i, y in enumerate(y_list):
        ax = axs[i]
        for j in range(y.shape[0]):
            color = cmap(norm(time_s[j]))
            ax.plot(x, y[j, :], color=color)

        # Place the title inside the plot, top left corner
        ax.text(0.03, 0.15, titles[i], transform=ax.transAxes, fontsize=22,
            verticalalignment='top', horizontalalignment='left', bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))
        ax.set_xlim(x.min(), x.max())
        ax.set_ylim(ylowerlimit, yupperlimit)
        ax.tick_params(axis='both', which='both', direction='in', top=True, right=True)
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        # if i % 3 == 0:
        #     ax.set_ylabel(r'$\log(n\, [\mathrm{cm}^{-3}])$')
        # if i >= 3:
        #     ax.set_xlabel(r'$\log(E\, [\mathrm{MeV}])$')

    fig.text(0.5, 0.02, r'$\log_{10}\, E\, [\mathrm{MeV}] $', ha='center', fontsize=22)
    fig.text(0.07, 0.5, r'$f-f^i $', va='center', rotation='vertical', fontsize=22)

    # Shared colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=axs, location='right', shrink=1.0, pad=0.01)
    cbar.set_label(r'$t \, [\mathrm{ms}]$', fontsize=22)
    cbar.ax.yaxis.set_minor_locator(AutoMinorLocator())

    # Super title
    fig.suptitle(suptitle, fontsize=22)

    # Save and show
    if filename:
        plt.savefig(filename, bbox_inches='tight')
    plt.show()
    plt.close(fig)

a=0
b=-485
c=None

plot_colored_lines_grid(
    x = np.log10(energybinsMeV),
    y_list = [
        all_f00_Re_split_energy_bins[a:b:c, :] - all_f00_Re_split_energy_bins[a],
        all_f11_Re_split_energy_bins[a:b:c, :] - all_f11_Re_split_energy_bins[a],
        all_f22_Re_split_energy_bins[a:b:c, :] - all_f22_Re_split_energy_bins[a],
        all_f00_Rebar_split_energy_bins[a:b:c, :] - all_f00_Rebar_split_energy_bins[a],
        all_f11_Rebar_split_energy_bins[a:b:c, :] - all_f11_Rebar_split_energy_bins[a],
        all_f22_Rebar_split_energy_bins[a:b:c, :] - all_f22_Rebar_split_energy_bins[a]
    ],
    time_s = tiempo_s[a:b:c] / 1e-3,
    titles = [
        r'$e$', r'$\mu$', r'$\tau$',
         r'$\bar{e}$', r'$\bar{\mu}$', r'$\bar{ \tau }$'
    ],
    suptitle = '',
    filename = f"plots/ffin_min_fini_CFI_E_spectrum.pdf",
    ylowerlimit = -0.019,
    yupperlimit = +0.031
)

### READ NUMBER AND ENERGY DENSITY

In [ ]:
with h5py.File(f"/home/erick/software/devscrpts/gw170817_paper_plots/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_10.0m_att_1e-5_n_average_time_evolution.h5", "r") as hf:
    time_s_1emin5 = hf["time_s"][:]
    n_ee_ave_inv_ccm_1emin5 = hf["n_ee_ave_inv_ccm"][:]
    n_uu_ave_inv_ccm_1emin5 = hf["n_uu_ave_inv_ccm"][:]
    n_tt_ave_inv_ccm_1emin5 = hf["n_tt_ave_inv_ccm"][:]
    n_eu_ave_inv_ccm_1emin5 = hf["n_eu_ave_inv_ccm"][:]
    n_et_ave_inv_ccm_1emin5 = hf["n_et_ave_inv_ccm"][:]
    n_ut_ave_inv_ccm_1emin5 = hf["n_ut_ave_inv_ccm"][:]
    nbar_ee_ave_inv_ccm_1emin5 = hf["nbar_ee_ave_inv_ccm"][:]
    nbar_uu_ave_inv_ccm_1emin5 = hf["nbar_uu_ave_inv_ccm"][:]
    nbar_tt_ave_inv_ccm_1emin5 = hf["nbar_tt_ave_inv_ccm"][:]
    nbar_eu_ave_inv_ccm_1emin5 = hf["nbar_eu_ave_inv_ccm"][:]
    nbar_et_ave_inv_ccm_1emin5 = hf["nbar_et_ave_inv_ccm"][:]
    nbar_ut_ave_inv_ccm_1emin5 = hf["nbar_ut_ave_inv_ccm"][:]

with h5py.File(f"/home/erick/software/devscrpts/gw170817_paper_plots/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_01.0m_att_1e-4_n_average_time_evolution.h5", "r") as hf:
    time_s_1emin4 = hf["time_s"][:]
    n_ee_ave_inv_ccm_1emin4 = hf["n_ee_ave_inv_ccm"][:]
    n_uu_ave_inv_ccm_1emin4 = hf["n_uu_ave_inv_ccm"][:]
    n_tt_ave_inv_ccm_1emin4 = hf["n_tt_ave_inv_ccm"][:]
    n_eu_ave_inv_ccm_1emin4 = hf["n_eu_ave_inv_ccm"][:]
    n_et_ave_inv_ccm_1emin4 = hf["n_et_ave_inv_ccm"][:]
    n_ut_ave_inv_ccm_1emin4 = hf["n_ut_ave_inv_ccm"][:]
    nbar_ee_ave_inv_ccm_1emin4 = hf["nbar_ee_ave_inv_ccm"][:]
    nbar_uu_ave_inv_ccm_1emin4 = hf["nbar_uu_ave_inv_ccm"][:]
    nbar_tt_ave_inv_ccm_1emin4 = hf["nbar_tt_ave_inv_ccm"][:]
    nbar_eu_ave_inv_ccm_1emin4 = hf["nbar_eu_ave_inv_ccm"][:]
    nbar_et_ave_inv_ccm_1emin4 = hf["nbar_et_ave_inv_ccm"][:]
    nbar_ut_ave_inv_ccm_1emin4 = hf["nbar_ut_ave_inv_ccm"][:]

with h5py.File(f"/home/erick/software/devscrpts/gw170817_paper_plots/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_00.1m_att_1e-3_n_average_time_evolution.h5", "r") as hf:
    time_s_1emin3 = hf["time_s"][:]
    n_ee_ave_inv_ccm_1emin3 = hf["n_ee_ave_inv_ccm"][:]
    n_uu_ave_inv_ccm_1emin3 = hf["n_uu_ave_inv_ccm"][:]
    n_tt_ave_inv_ccm_1emin3 = hf["n_tt_ave_inv_ccm"][:]
    n_eu_ave_inv_ccm_1emin3 = hf["n_eu_ave_inv_ccm"][:]
    n_et_ave_inv_ccm_1emin3 = hf["n_et_ave_inv_ccm"][:]
    n_ut_ave_inv_ccm_1emin3 = hf["n_ut_ave_inv_ccm"][:]
    nbar_ee_ave_inv_ccm_1emin3 = hf["nbar_ee_ave_inv_ccm"][:]
    nbar_uu_ave_inv_ccm_1emin3 = hf["nbar_uu_ave_inv_ccm"][:]
    nbar_tt_ave_inv_ccm_1emin3 = hf["nbar_tt_ave_inv_ccm"][:]
    nbar_eu_ave_inv_ccm_1emin3 = hf["nbar_eu_ave_inv_ccm"][:]
    nbar_et_ave_inv_ccm_1emin3 = hf["nbar_et_ave_inv_ccm"][:]
    nbar_ut_ave_inv_ccm_1emin3 = hf["nbar_ut_ave_inv_ccm"][:]

with h5py.File(f"/home/erick/software/devscrpts/gw170817_paper_plots/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_10.0m_att_1e-5_energy_density_time_evolution.h5", "r") as hf:
    tiempo_s = hf["tiempo_s"][:]
    energy_n00 = hf["energy_n00"][:]
    energy_n00bar = hf["energy_n00bar"][:]
    energy_n11 = hf["energy_n11"][:]
    energy_n11bar = hf["energy_n11bar"][:]
    energy_n22 = hf["energy_n22"][:]
    energy_n22bar = hf["energy_n22bar"][:]

### PLOT NUMBER DENSITY

In [ ]:
fig, (ax_1, ax_2, ax_3, ax_4, ax_5) = plt.subplots(
    5, 1, sharex=True, figsize=(10, 30),
    gridspec_kw=dict(hspace=0.0) # Gap panel above and below
)

##############################################################################

nor_val_y=1.0e32
nor_val_x=1.0e-3

ax_1.plot(time_s_1emin5/nor_val_x, n_ee_ave_inv_ccm_1emin5/nor_val_y, color = 'C0', linestyle='solid', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin5/nor_val_x, n_uu_ave_inv_ccm_1emin5/nor_val_y, color = 'C1', linestyle='solid', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin5/nor_val_x, n_tt_ave_inv_ccm_1emin5/nor_val_y, color = 'C2', linestyle='solid', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin4/nor_val_x, n_ee_ave_inv_ccm_1emin4/nor_val_y, color = 'C0', linestyle='dashed', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin4/nor_val_x, n_uu_ave_inv_ccm_1emin4/nor_val_y, color = 'C1', linestyle='dashed', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin4/nor_val_x, n_tt_ave_inv_ccm_1emin4/nor_val_y, color = 'C2', linestyle='dashed', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin3/nor_val_x, n_ee_ave_inv_ccm_1emin3/nor_val_y, color = 'C0', linestyle='dotted', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin3/nor_val_x, n_uu_ave_inv_ccm_1emin3/nor_val_y, color = 'C1', linestyle='dotted', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin3/nor_val_x, n_tt_ave_inv_ccm_1emin3/nor_val_y, color = 'C2', linestyle='dotted', linewidth=3, alpha=1.0)

ax_1.set_ylabel(r'$n \, [10^{32}\,\mathrm{cm}^{-3}]$',fontsize=30)
ax_1.tick_params(axis='y', labelsize=30)

handles_1_1 = [
    (
        Line2D([], [], color='C0', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C0', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C0', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C1', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C2', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C0', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='solid', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C0', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dashed', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C0', linestyle='dotted', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dotted', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dotted', lw=3, alpha=1.0),
    ),
]
labels_1_1  = [
    r'$n_{ee}$',
    r'$n_{\mu\mu}$',
    r'$n_{\tau\tau}$',
    r'$\eta=10^{-5}$',
    r'$\eta=10^{-4}$',
    r'$\eta=10^{-3}$',
]
# first legend: injection lines only –- make them long
leg_1_1 = ax_1.legend(
    handles_1_1, labels_1_1,
    handler_map={tuple: HandlerTuple(ndivide=None)},
    handlelength=5.0,        # ← long lines here
    ncol=2, fontsize=20, framealpha=0.0,
    loc='upper left',        # base location
    bbox_to_anchor=(0.0, 0.80)
)

ax_1.add_artist(leg_1_1)        # keep it when we add the second legend
pf.apply_custom_settings(ax_1, leg_1_1, log_scale_y=False)
# ax_1.grid(True, which='both', linestyle=':', linewidth=1, alpha=0.8)

##############################################################################

nor_val_y=1.0
nor_val_x=1.0e-3

ax_2.plot(time_s_1emin5/nor_val_x, n_eu_ave_inv_ccm_1emin5/nor_val_y, color = 'C3', linestyle='solid', linewidth=3, alpha=1.0, label=r'$n_{e\mu}$')
ax_2.plot(time_s_1emin5/nor_val_x, n_et_ave_inv_ccm_1emin5/nor_val_y, color = 'C4', linestyle='solid', linewidth=3, alpha=1.0, label=r'$n_{e\tau}$')
ax_2.plot(time_s_1emin5/nor_val_x, n_ut_ave_inv_ccm_1emin5/nor_val_y, color = 'C5', linestyle='solid', linewidth=3, alpha=1.0, label=r'$n_{\mu\tau}$')
ax_2.plot(time_s_1emin4/nor_val_x, n_eu_ave_inv_ccm_1emin4/nor_val_y, color = 'C3', linestyle='dashed', linewidth=3, alpha=1.0)
ax_2.plot(time_s_1emin4/nor_val_x, n_et_ave_inv_ccm_1emin4/nor_val_y, color = 'C4', linestyle='dashed', linewidth=3, alpha=1.0)
ax_2.plot(time_s_1emin4/nor_val_x, n_ut_ave_inv_ccm_1emin4/nor_val_y, color = 'C5', linestyle='dashed', linewidth=3, alpha=1.0)
ax_2.plot(time_s_1emin3/nor_val_x, n_eu_ave_inv_ccm_1emin3/nor_val_y, color = 'C3', linestyle='dotted', linewidth=3, alpha=1.0)
ax_2.plot(time_s_1emin3/nor_val_x, n_et_ave_inv_ccm_1emin3/nor_val_y, color = 'C4', linestyle='dotted', linewidth=3, alpha=1.0)
ax_2.plot(time_s_1emin3/nor_val_x, n_ut_ave_inv_ccm_1emin3/nor_val_y, color = 'C5', linestyle='dotted', linewidth=3, alpha=1.0)


#####################################
# Plotting most unstable mode
Omega_inv_s = 2.068e4 # in 1/s

A = 5e28 / np.exp(Omega_inv_s * 0.2e-3)
time_LSA = np.linspace(0.2, 0.5, 20)*nor_val_x  # in s
print("time_LSA =", time_LSA)
y = A * np.exp(Omega_inv_s * time_LSA)
print("y =", y)
print("A =", A)
ax_2.plot(time_LSA/nor_val_x, y/nor_val_y, color='black', linestyle='solid', linewidth=2.0, alpha=1.0)
fig.text(
    0.138, 0.67,
    r'$\mathrm{Im} \, \Omega^{\mathrm{max}} = 2.068\times 10^{4} \,\mathrm{s}^{-1}$',
    va='center', color='black', rotation=73, fontsize=20
)

#####################################

ax_2.set_ylabel(r'$n \, [\mathrm{cm}^{-3}]$', fontsize=30)
ax_2.tick_params(axis='y', labelsize=30)

handles_2_1 = [
    (
        Line2D([], [], color='C3', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C3', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C3', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C4', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C5', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C3', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='solid', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C3', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dashed', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C3', linestyle='dotted', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dotted', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dotted', lw=3, alpha=1.0),
    ),
]
labels_2_1  = [
    r'$n_{e\mu}$',
    r'$n_{e\tau}$',
    r'$n_{\mu\tau}$',
    r'$\eta=10^{-5}$',
    r'$\eta=10^{-4}$',
    r'$\eta=10^{-3}$',
]
# first legend: injection lines only –- make them long
leg_2_1 = ax_2.legend(
    handles_2_1, labels_2_1,
    handler_map={tuple: HandlerTuple(ndivide=None)},
    handlelength=5.0,        # ← long lines here
    ncol=2, fontsize=20, framealpha=0.0,
    loc='lower right',        # base location
    # bbox_to_anchor=(0.10, 0.80)
)

ax_2.add_artist(leg_2_1)        # keep it when we add the second legend
pf.apply_custom_settings(ax_2, leg_2_1, log_scale_y=True)
ax_2.set_ylim(1e27, 2e32)
# ax_2.grid(True, which='both', linestyle=':', linewidth=1, alpha=0.8)

##############################################################################

nor_val_y=1.0e32
nor_val_x=1.0e-3

ax_3.plot(time_s_1emin5/nor_val_x, nbar_ee_ave_inv_ccm_1emin5/nor_val_y, color = 'C0', linestyle='solid', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin5/nor_val_x, nbar_uu_ave_inv_ccm_1emin5/nor_val_y, color = 'C1', linestyle='solid', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin5/nor_val_x, nbar_tt_ave_inv_ccm_1emin5/nor_val_y, color = 'C2', linestyle='solid', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin4/nor_val_x, nbar_ee_ave_inv_ccm_1emin4/nor_val_y, color = 'C0', linestyle='dashed', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin4/nor_val_x, nbar_uu_ave_inv_ccm_1emin4/nor_val_y, color = 'C1', linestyle='dashed', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin4/nor_val_x, nbar_tt_ave_inv_ccm_1emin4/nor_val_y, color = 'C2', linestyle='dashed', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin3/nor_val_x, nbar_ee_ave_inv_ccm_1emin3/nor_val_y, color = 'C0', linestyle='dotted', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin3/nor_val_x, nbar_uu_ave_inv_ccm_1emin3/nor_val_y, color = 'C1', linestyle='dotted', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin3/nor_val_x, nbar_tt_ave_inv_ccm_1emin3/nor_val_y, color = 'C2', linestyle='dotted', linewidth=3, alpha=1.0)

ax_3.set_ylabel(r'$\bar{n} \, [10^{32}\,\mathrm{cm}^{-3}]$',fontsize=30)
ax_3.tick_params(axis='y', labelsize=30)

handles_3_1 = [
    (
        Line2D([], [], color='C0', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C0', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C0', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C1', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C2', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dotted', lw=3, alpha=1.0),
    ),
]
labels_3_1  = [
    r'$\bar{n}_{ee}$',
    r'$\bar{n}_{\mu\mu}$',
    r'$\bar{n}_{\tau\tau}$',
]
leg_3_1 = ax_3.legend(
    handles_3_1, labels_3_1,
    handler_map={tuple: HandlerTuple(ndivide=None)},
    handlelength=5.0,
    ncol=3, fontsize=20, framealpha=0.0,
    loc='upper center',
    # bbox_to_anchor=(0.10, 0.80)
)
ax_3.set_ylim(None,3.6)
ax_3.add_artist(leg_3_1)
pf.apply_custom_settings(ax_3, leg_3_1, log_scale_y=False)

##############################################################################

ax_4.plot(time_s_1emin5/1e-3, nbar_eu_ave_inv_ccm_1emin5, color = 'C3', linestyle='solid', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin5/1e-3, nbar_et_ave_inv_ccm_1emin5, color = 'C4', linestyle='solid', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin5/1e-3, nbar_ut_ave_inv_ccm_1emin5, color = 'C5', linestyle='solid', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin4/1e-3, nbar_eu_ave_inv_ccm_1emin4, color = 'C3', linestyle='dashed', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin4/1e-3, nbar_et_ave_inv_ccm_1emin4, color = 'C4', linestyle='dashed', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin4/1e-3, nbar_ut_ave_inv_ccm_1emin4, color = 'C5', linestyle='dashed', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin3/1e-3, nbar_eu_ave_inv_ccm_1emin3, color = 'C3', linestyle='dotted', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin3/1e-3, nbar_et_ave_inv_ccm_1emin3, color = 'C4', linestyle='dotted', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin3/1e-3, nbar_ut_ave_inv_ccm_1emin3, color = 'C5', linestyle='dotted', linewidth=3, alpha=1.0)

ax_4.set_ylim(1e27, 2e32)
ax_4.set_ylabel(r'$\bar{n} \, [\mathrm{cm}^{-3}]$', fontsize=30)
ax_4.tick_params(axis='y', labelsize=30)

handles_4_1 = [
    (
        Line2D([], [], color='C3', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C3', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C3', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C4', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C5', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dotted', lw=3, alpha=1.0),
    ),
]
labels_4_1  = [
    r'$\bar{n}_{e\mu}$',
    r'$\bar{n}_{e\tau}$',
    r'$\bar{n}_{\mu\tau}$',
]
leg_4_1 = ax_4.legend(
    handles_4_1, labels_4_1,
    handler_map={tuple: HandlerTuple(ndivide=None)},
    handlelength=5.0,
    ncol=1, fontsize=20, framealpha=0.0,
    loc='lower right',
    # bbox_to_anchor=(0.10, 0.80)
)
ax_4.add_artist(leg_4_1)
pf.apply_custom_settings(ax_4, leg_4_1, log_scale_y=True)

##############################################################################

nor_val_y=1.0e34
nor_val_x=1.0e-3

ax_5.plot(tiempo_s/nor_val_x, energy_n00/nor_val_y, label=r'$\epsilon_{e}$', linewidth=3)
ax_5.plot(tiempo_s/nor_val_x, energy_n11/nor_val_y, label=r'$\epsilon_{\mu}$', linewidth=3)
ax_5.plot(tiempo_s/nor_val_x, energy_n22/nor_val_y, label=r'$\epsilon_{\tau}$', linewidth=3)
ax_5.plot(tiempo_s/nor_val_x, energy_n00bar/nor_val_y, label=r'$\bar{\epsilon}_{e}$', linewidth=3)
ax_5.plot(tiempo_s/nor_val_x, energy_n11bar/nor_val_y, label=r'$\bar{\epsilon}_{\mu}$', linewidth=3)
ax_5.plot(tiempo_s/nor_val_x, energy_n22bar/nor_val_y, label=r'$\bar{\epsilon}_{\tau}$', linewidth=3)

ax_5.set_xlim(0.0, 3.2)
ax_5.set_ylim(-0.05, 1.15)
leg_5 = ax_5.legend(framealpha=0.0, ncol=2, fontsize=20, loc='lower right')
pf.apply_custom_settings(ax_5, leg_5, False)
ax_5.add_artist(leg_5)
ax_5.set_ylabel(r'$\epsilon \, [10^{34}\, \mathrm{MeV} \, \mathrm{cm}^{-3} ]$', fontsize=30)
ax_5.set_xlabel(r'$t \, [\mathrm{ms}]$', style="italic", fontsize=30)
ax_5.tick_params(axis='y', labelsize=30)
ax_5.tick_params(axis='x', labelsize=30)

##############################################################################

plt.savefig('plots/n_CFI.pdf', bbox_inches='tight')
plt.show()
plt.close(fig)

### Read flux number densities

In [ ]:
with h5py.File(f"/home/erick/software/devscrpts/gw170817_paper_plots/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_10.0m_att_1e-5_flux_average_time_evolution.h5", "r") as hf:
    time_s_1emin5 = hf["time_s"][:]
    F00_ave_inv_ccm_1emin5 = hf["F00_ave_inv_ccm"][:]
    F11_ave_inv_ccm_1emin5 = hf["F11_ave_inv_ccm"][:]
    F22_ave_inv_ccm_1emin5 = hf["F22_ave_inv_ccm"][:]
    F00bar_ave_inv_ccm_1emin5 = hf["F00bar_ave_inv_ccm"][:]
    F11bar_ave_inv_ccm_1emin5 = hf["F11bar_ave_inv_ccm"][:]
    F22bar_ave_inv_ccm_1emin5 = hf["F22bar_ave_inv_ccm"][:]
    F01_ave_inv_ccm_1emin5 = hf["F01_ave_inv_ccm"][:]
    F02_ave_inv_ccm_1emin5 = hf["F02_ave_inv_ccm"][:]
    F12_ave_inv_ccm_1emin5 = hf["F12_ave_inv_ccm"][:]
    F01bar_ave_inv_ccm_1emin5 = hf["F01bar_ave_inv_ccm"][:]
    F02bar_ave_inv_ccm_1emin5 = hf["F02bar_ave_inv_ccm"][:]
    F12bar_ave_inv_ccm_1emin5 = hf["F12bar_ave_inv_ccm"][:]

with h5py.File(f"/home/erick/software/devscrpts/gw170817_paper_plots/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_01.0m_att_1e-4_flux_average_time_evolution.h5", "r") as hf:
    time_s_1emin4 = hf["time_s"][:]
    F00_ave_inv_ccm_1emin4 = hf["F00_ave_inv_ccm"][:]
    F11_ave_inv_ccm_1emin4 = hf["F11_ave_inv_ccm"][:]
    F22_ave_inv_ccm_1emin4 = hf["F22_ave_inv_ccm"][:]
    F01_ave_inv_ccm_1emin4 = hf["F01_ave_inv_ccm"][:]
    F02_ave_inv_ccm_1emin4 = hf["F02_ave_inv_ccm"][:]
    F12_ave_inv_ccm_1emin4 = hf["F12_ave_inv_ccm"][:]
    F00bar_ave_inv_ccm_1emin4 = hf["F00bar_ave_inv_ccm"][:]
    F11bar_ave_inv_ccm_1emin4 = hf["F11bar_ave_inv_ccm"][:]
    F22bar_ave_inv_ccm_1emin4 = hf["F22bar_ave_inv_ccm"][:]
    F01bar_ave_inv_ccm_1emin4 = hf["F01bar_ave_inv_ccm"][:]
    F02bar_ave_inv_ccm_1emin4 = hf["F02bar_ave_inv_ccm"][:]
    F12bar_ave_inv_ccm_1emin4 = hf["F12bar_ave_inv_ccm"][:]

with h5py.File(f"/home/erick/software/devscrpts/gw170817_paper_plots/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_00.1m_att_1e-3_flux_average_time_evolution.h5", "r") as hf:
    time_s_1emin3 = hf["time_s"][:]
    F00_ave_inv_ccm_1emin3 = hf["F00_ave_inv_ccm"][:]
    F11_ave_inv_ccm_1emin3 = hf["F11_ave_inv_ccm"][:]
    F22_ave_inv_ccm_1emin3 = hf["F22_ave_inv_ccm"][:]
    F01_ave_inv_ccm_1emin3 = hf["F01_ave_inv_ccm"][:]
    F02_ave_inv_ccm_1emin3 = hf["F02_ave_inv_ccm"][:]
    F12_ave_inv_ccm_1emin3 = hf["F12_ave_inv_ccm"][:]
    F00bar_ave_inv_ccm_1emin3 = hf["F00bar_ave_inv_ccm"][:]
    F11bar_ave_inv_ccm_1emin3 = hf["F11bar_ave_inv_ccm"][:]
    F22bar_ave_inv_ccm_1emin3 = hf["F22bar_ave_inv_ccm"][:]
    F01bar_ave_inv_ccm_1emin3 = hf["F01bar_ave_inv_ccm"][:]
    F02bar_ave_inv_ccm_1emin3 = hf["F02bar_ave_inv_ccm"][:]
    F12bar_ave_inv_ccm_1emin3 = hf["F12bar_ave_inv_ccm"][:]

In [ ]:

fig, (ax_1, ax_2, ax_3, ax_4) = plt.subplots(
    4, 1, sharex=True, figsize=(10, 25),
    gridspec_kw=dict(hspace=0.0) # Gap panel above and below
)

##############################################################################

nor_val_y=1.0e32
nor_val_x=1.0e-3

ax_1.plot(time_s_1emin5/nor_val_x, F00_ave_inv_ccm_1emin5/nor_val_y, color = 'C0', linestyle='solid', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin5/nor_val_x, F11_ave_inv_ccm_1emin5/nor_val_y, color = 'C1', linestyle='solid', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin5/nor_val_x, F22_ave_inv_ccm_1emin5/nor_val_y, color = 'C2', linestyle='solid', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin4/nor_val_x, F00_ave_inv_ccm_1emin4/nor_val_y, color = 'C0', linestyle='dashed', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin4/nor_val_x, F11_ave_inv_ccm_1emin4/nor_val_y, color = 'C1', linestyle='dashed', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin4/nor_val_x, F22_ave_inv_ccm_1emin4/nor_val_y, color = 'C2', linestyle='dashed', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin3/nor_val_x, F00_ave_inv_ccm_1emin3/nor_val_y, color = 'C0', linestyle='dotted', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin3/nor_val_x, F11_ave_inv_ccm_1emin3/nor_val_y, color = 'C1', linestyle='dotted', linewidth=3, alpha=1.0)
ax_1.plot(time_s_1emin3/nor_val_x, F22_ave_inv_ccm_1emin3/nor_val_y, color = 'C2', linestyle='dotted', linewidth=3, alpha=1.0)

ax_1.set_ylabel(r'$|\vec{f}| \, [10^{32}\,\mathrm{cm}^{-3}]$',fontsize=30)
ax_1.tick_params(axis='y', labelsize=30)

handles_1_1 = [
    (
        Line2D([], [], color='C0', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C0', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C0', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C1', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C2', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C0', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='solid', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C0', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dashed', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C0', linestyle='dotted', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dotted', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dotted', lw=3, alpha=1.0),
    ),
]
labels_1_1  = [
    r'$|\vec{f}_{ee}|$',
    r'$|\vec{f}_{\mu\mu}|$',
    r'$|\vec{f}_{\tau\tau}|$',
    r'$\eta=10^{-5}$',
    r'$\eta=10^{-4}$',
    r'$\eta=10^{-3}$',
]
# first legend: injection lines only –- make them long
leg_1_1 = ax_1.legend(
    handles_1_1, labels_1_1,
    handler_map={tuple: HandlerTuple(ndivide=None)},
    handlelength=5.0,        # ← long lines here
    ncol=1, fontsize=20, framealpha=0.0,
    loc='upper left',        # base location
    bbox_to_anchor=(0.6, 1.0)
)

ax_1.set_ylim(-0.26/10,0.26)
ax_1.add_artist(leg_1_1)        # keep it when we add the second legend
pf.apply_custom_settings(ax_1, leg_1_1, log_scale_y=False)
# ax_1.grid(True, which='both', linestyle=':', linewidth=1, alpha=0.8)

##############################################################################

nor_val_y=1.0
nor_val_x=1.0e-3

ax_2.plot(time_s_1emin5/nor_val_x, F01_ave_inv_ccm_1emin5/nor_val_y, color = 'C3', linestyle='solid', linewidth=3, alpha=1.0, label=r'$|\vec{f}_{e\mu}|$')
ax_2.plot(time_s_1emin5/nor_val_x, F02_ave_inv_ccm_1emin5/nor_val_y, color = 'C4', linestyle='solid', linewidth=3, alpha=1.0, label=r'$|\vec{f}_{e\tau}|$')
ax_2.plot(time_s_1emin5/nor_val_x, F12_ave_inv_ccm_1emin5/nor_val_y, color = 'C5', linestyle='solid', linewidth=3, alpha=1.0, label=r'$|\vec{f}_{\mu\tau}|$')
ax_2.plot(time_s_1emin4/nor_val_x, F01_ave_inv_ccm_1emin4/nor_val_y, color = 'C3', linestyle='dashed', linewidth=3, alpha=1.0)
ax_2.plot(time_s_1emin4/nor_val_x, F02_ave_inv_ccm_1emin4/nor_val_y, color = 'C4', linestyle='dashed', linewidth=3, alpha=1.0)
ax_2.plot(time_s_1emin4/nor_val_x, F12_ave_inv_ccm_1emin4/nor_val_y, color = 'C5', linestyle='dashed', linewidth=3, alpha=1.0)
ax_2.plot(time_s_1emin3/nor_val_x, F01_ave_inv_ccm_1emin3/nor_val_y, color = 'C3', linestyle='dotted', linewidth=3, alpha=1.0)
ax_2.plot(time_s_1emin3/nor_val_x, F02_ave_inv_ccm_1emin3/nor_val_y, color = 'C4', linestyle='dotted', linewidth=3, alpha=1.0)
ax_2.plot(time_s_1emin3/nor_val_x, F12_ave_inv_ccm_1emin3/nor_val_y, color = 'C5', linestyle='dotted', linewidth=3, alpha=1.0)


#####################################
# Plotting most unstable mode
Omega_inv_s = 2.068e4 # in 1/s

A = 5e28 / np.exp(Omega_inv_s * 0.2e-3)
time_LSA = np.linspace(0.2, 0.5, 20)*nor_val_x  # in s
print("time_LSA =", time_LSA)
y = A * np.exp(Omega_inv_s * time_LSA)
print("y =", y)
print("A =", A)
ax_2.plot(time_LSA/nor_val_x, y/nor_val_y, color='black', linestyle='solid', linewidth=2.0, alpha=1.0)
fig.text(
    0.138, 0.62,
    r'$\mathrm{Im} \, \Omega^{\mathrm{max}} = 2.068\times 10^{4} \,\mathrm{s}^{-1}$',
    va='center', color='black', rotation=73, fontsize=20
)

#####################################

ax_2.set_ylabel(r'$|\vec{f}| \, [\mathrm{cm}^{-3}]$', fontsize=30)
ax_2.tick_params(axis='y', labelsize=30)

handles_2_1 = [
    (
        Line2D([], [], color='C3', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C3', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C3', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C4', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C5', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C3', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='solid', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C3', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dashed', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C3', linestyle='dotted', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dotted', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dotted', lw=3, alpha=1.0),
    ),
]
labels_2_1  = [
    r'$|\vec{f}_{e\mu}|$',
    r'$|\vec{f}_{e\tau}|$',
    r'$|\vec{f}_{\mu\tau}|$',
    r'$\eta=10^{-5}$',
    r'$\eta=10^{-4}$',
    r'$\eta=10^{-3}$',
]
# first legend: injection lines only –- make them long
leg_2_1 = ax_2.legend(
    handles_2_1, labels_2_1,
    handler_map={tuple: HandlerTuple(ndivide=None)},
    handlelength=5.0,        # ← long lines here
    ncol=2, fontsize=20, framealpha=0.0,
    loc='lower right',        # base location
    # bbox_to_anchor=(0.10, 0.80)
)

ax_2.add_artist(leg_2_1)        # keep it when we add the second legend
pf.apply_custom_settings(ax_2, leg_2_1, log_scale_y=True)
ax_2.set_ylim(1e27, 2e32)
# ax_2.grid(True, which='both', linestyle=':', linewidth=1, alpha=0.8)

##############################################################################

nor_val_y=1.0e32
nor_val_x=1.0e-3

ax_3.plot(time_s_1emin5/nor_val_x, F00bar_ave_inv_ccm_1emin5/nor_val_y, color = 'C0', linestyle='solid', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin5/nor_val_x, F11bar_ave_inv_ccm_1emin5/nor_val_y, color = 'C1', linestyle='solid', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin5/nor_val_x, F22bar_ave_inv_ccm_1emin5/nor_val_y, color = 'C2', linestyle='solid', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin4/nor_val_x, F00bar_ave_inv_ccm_1emin4/nor_val_y, color = 'C0', linestyle='dashed', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin4/nor_val_x, F11bar_ave_inv_ccm_1emin4/nor_val_y, color = 'C1', linestyle='dashed', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin4/nor_val_x, F22bar_ave_inv_ccm_1emin4/nor_val_y, color = 'C2', linestyle='dashed', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin3/nor_val_x, F00bar_ave_inv_ccm_1emin3/nor_val_y, color = 'C0', linestyle='dotted', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin3/nor_val_x, F11bar_ave_inv_ccm_1emin3/nor_val_y, color = 'C1', linestyle='dotted', linewidth=3, alpha=1.0)
ax_3.plot(time_s_1emin3/nor_val_x, F22bar_ave_inv_ccm_1emin3/nor_val_y, color = 'C2', linestyle='dotted', linewidth=3, alpha=1.0)

ax_3.set_ylabel(r'$|\vec{f}| \, [10^{32}\,\mathrm{cm}^{-3}]$',fontsize=30)
ax_3.tick_params(axis='y', labelsize=30)

handles_3_1 = [
    (
        Line2D([], [], color='C0', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C0', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C0', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C1', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C1', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C2', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C2', linestyle='dotted', lw=3, alpha=1.0),
    ),
]
labels_3_1  = [
    r'$|\vec{f}_{ee}|$',
    r'$|\vec{f}_{\mu\mu}|$',
    r'$|\vec{f}_{\tau\tau}|$',
]
leg_3_1 = ax_3.legend(
    handles_3_1, labels_3_1,
    handler_map={tuple: HandlerTuple(ndivide=None)},
    handlelength=5.0,
    ncol=1, fontsize=20, framealpha=0.0,
    loc='upper right',
    # bbox_to_anchor=(0.10, 0.80)
)
ax_3.set_ylim(-0.26/10,0.26)
ax_3.add_artist(leg_3_1)
pf.apply_custom_settings(ax_3, leg_3_1, log_scale_y=False)

##############################################################################

ax_4.plot(time_s_1emin5/1e-3, F01bar_ave_inv_ccm_1emin5, color = 'C3', linestyle='solid', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin5/1e-3, F02bar_ave_inv_ccm_1emin5, color = 'C4', linestyle='solid', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin5/1e-3, F12bar_ave_inv_ccm_1emin5, color = 'C5', linestyle='solid', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin4/1e-3, F01bar_ave_inv_ccm_1emin4, color = 'C3', linestyle='dashed', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin4/1e-3, F02bar_ave_inv_ccm_1emin4, color = 'C4', linestyle='dashed', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin4/1e-3, F12bar_ave_inv_ccm_1emin4, color = 'C5', linestyle='dashed', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin3/1e-3, F01bar_ave_inv_ccm_1emin3, color = 'C3', linestyle='dotted', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin3/1e-3, F02bar_ave_inv_ccm_1emin3, color = 'C4', linestyle='dotted', linewidth=3, alpha=1.0)
ax_4.plot(time_s_1emin3/1e-3, F12bar_ave_inv_ccm_1emin3, color = 'C5', linestyle='dotted', linewidth=3, alpha=1.0)

ax_4.set_ylim(1e27, 2e32)
ax_4.set_ylabel(r'$|\vec{f}| \, [\mathrm{cm}^{-3}]$', fontsize=30)
ax_4.tick_params(axis='y', labelsize=30)

handles_4_1 = [
    (
        Line2D([], [], color='C3', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C3', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C3', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C4', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C4', linestyle='dotted', lw=3, alpha=1.0),
    ),
    (
        Line2D([], [], color='C5', linestyle='solid', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dashed', lw=3, alpha=1.0),
        Line2D([], [], color='C5', linestyle='dotted', lw=3, alpha=1.0),
    ),
]
labels_4_1  = [
    r'$|\vec{f}_{e\mu}|$',
    r'$|\vec{f}_{e\tau}|$',
    r'$|\vec{f}_{\mu\tau}|$',
]
leg_4_1 = ax_4.legend(
    handles_4_1, labels_4_1,
    handler_map={tuple: HandlerTuple(ndivide=None)},
    handlelength=5.0,
    ncol=1, fontsize=20, framealpha=0.0,
    loc='lower right',
    # bbox_to_anchor=(0.10, 0.80)
)
ax_4.add_artist(leg_4_1)
pf.apply_custom_settings(ax_4, leg_4_1, log_scale_y=True)
ax_4.set_xlim(0.0, 3.2)
ax_4.set_xlabel(r'$t \, [\mathrm{ms}]$', style="italic", fontsize=30)

##############################################################################

plt.savefig('plots/f_CFI.pdf', bbox_inches='tight')
plt.show()
plt.close(fig)